In [1]:
import pandas as pd
import sqlite3

In [2]:
# This is the directory where you cloned the repo
path_to_repo = '/Users/ljob/Desktop/'

# Path to download data to
dir = path_to_repo + 'cnbs-predictor/data/'

# Read in the CSVs
tmp_csv = dir + 'CFS_TMP_forecasts_Avgs_K.csv'
evap_csv = dir + 'CFS_EVAP_forecasts_Avgs_MM.csv'
pcp_csv = dir + 'CFS_PCP_forecasts_Avgs_MM.csv'

In [3]:
## Read in PCP data from CFSR [mm]
pcp_data = pd.read_csv(pcp_csv,sep=',')

## Read in EVAP data from CFSR [mm]
evap_data = pd.read_csv(evap_csv,sep=',')

## Read in TMP data from CFSR [K]
tmp_data = pd.read_csv(tmp_csv,sep=',')

In [4]:
X = pd.DataFrame({
    'cfs_run': pcp_data['cfs_run'],
    'forecast_month': pcp_data['forecast_month'],
    'forecast_year': pcp_data['forecast_year'],
    'sup_pcp_lake': pcp_data['sup_lake'],
    'eri_pcp_lake': pcp_data['eri_lake'],
    'ont_pcp_lake': pcp_data['ont_lake'],
    'mih_pcp_lake': pcp_data['mih_lake'],
    'sup_pcp_land': pcp_data['sup_land'],
    'eri_pcp_land': pcp_data['eri_land'],
    'ont_pcp_land': pcp_data['ont_land'],
    'mih_pcp_land': pcp_data['mih_land'],
    'sup_evap_lake': evap_data['sup_lake'],
    'eri_evap_lake': evap_data['eri_lake'],
    'ont_evap_lake': evap_data['ont_lake'],
    'mih_evap_lake': evap_data['mih_lake'],
    'sup_evap_land': evap_data['sup_land'],
    'eri_evap_land': evap_data['eri_land'],
    'ont_evap_land': evap_data['ont_land'],
    'mih_evap_land': evap_data['mih_land'],
    'sup_tmp_lake': tmp_data['sup_lake'],
    'eri_tmp_lake': tmp_data['eri_lake'],
    'ont_tmp_lake': tmp_data['ont_lake'],
    'mih_tmp_lake': tmp_data['mih_lake'],
    'sup_tmp_land': tmp_data['sup_land'],
    'eri_tmp_land': tmp_data['eri_land'],
    'ont_tmp_land': tmp_data['ont_land'],
    'mih_tmp_land': tmp_data['mih_land']
})

In [5]:
# Establish a connection to a new SQLite database
conn = sqlite3.connect('cfs_forecast_data.db')  # Create a new database file
cursor = conn.cursor()

# Create the table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS cfs_forecast_data (
    cfs_run INTEGER,
    year INTEGER,
    month INTEGER,
    lake TEXT,
    surface_type TEXT,
    cnbs TEXT,
    value REAL,
    PRIMARY KEY (cfs_run, year, month, lake, surface_type, cnbs)
)
''')

In [10]:
# Define a function to map the lake and cnbs values as per your instructions
def map_values(lake, cnbs):
    # Mapping lakes
    lake_mapping = {
        'eri': 'erie',
        'ont': 'ontario',
        'sup': 'superior',
        'mih': 'michigan-huron'
    }
    
    # Mapping cnbs
    cnbs_mapping = {
        'tmp': 'air_temperature',
        'pcp': 'precipitation',
        'evap': 'evaporation'
    }

    # Return mapped values or the original if not found
    mapped_lake = lake_mapping.get(lake, lake)
    mapped_cnbs = cnbs_mapping.get(cnbs, cnbs)
    
    return mapped_lake, mapped_cnbs

# Define a function to insert data into the database
def insert_data_to_db(df):
    # Iterate through each row of the dataframe
    for index, row in df.iterrows():
        # Extract the values from the row
        cfs_run = row['cfs_run']
        forecast_year = row['forecast_year']
        forecast_month = row['forecast_month']
        
        # Define the lake and surface_type columns that follow the 'lake'_'cnbs'_'surface_type' pattern
        for lake_cnbs_surface in df.columns[3:]:  # Skipping the first three columns ('cfs_run', 'forecast_year', 'forecast_month')
            # Split the column name into lake, cnbs, and surface_type
            lake, cnbs, surface_type = lake_cnbs_surface.split('_')
            
            # Map lake and cnbs to the new values
            mapped_lake, mapped_cnbs = map_values(lake, cnbs)
            
            # Get the value for this specific lake/cnbs/surface_type combination
            value = row[lake_cnbs_surface]

            # Prepare the SQL query
            query = '''
                INSERT INTO cfs_forecast_data (cfs_run, year, month, lake, surface_type, cnbs, value)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            '''
            
            # Execute the query with the values
            cursor.execute(query, (cfs_run, forecast_year, forecast_month, mapped_lake, surface_type, mapped_cnbs, value))
        
    # Commit the transaction to save the changes
    conn.commit()

In [11]:
# Call the function to insert the data
insert_data_to_db(X)

# Close the connection
conn.close()

In [13]:
# Connect to the SQLite database
conn = sqlite3.connect('cfs_forecast_data.db')

# Query the data into a pandas DataFrame
df = pd.read_sql_query("SELECT * FROM cfs_forecast_data;", conn)

print(df)

conn.close()

           cfs_run  year  month            lake surface_type             cnbs  \
0       2024010100  2024      1        superior         lake    precipitation   
1       2024010100  2024      1            erie         lake    precipitation   
2       2024010100  2024      1         ontario         lake    precipitation   
3       2024010100  2024      1  michigan-huron         lake    precipitation   
4       2024010100  2024      1        superior         land    precipitation   
...            ...   ...    ...             ...          ...              ...   
417547  2025030418  2025     12  michigan-huron         lake  air_temperature   
417548  2025030418  2025     12        superior         land  air_temperature   
417549  2025030418  2025     12            erie         land  air_temperature   
417550  2025030418  2025     12         ontario         land  air_temperature   
417551  2025030418  2025     12  michigan-huron         land  air_temperature   

             value  
0     